In [0]:
!pip install deeppavlov

In [0]:
!pip install tensorflow==1.15.0

In [0]:
!apt-get --yes install git

In [0]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
from google.colab import files
uploaded = files.upload()

In [42]:
!ls *.txt

test.txt  train.txt  valid.txt


In [0]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  
ner_model = train_model(ner_config, download=True)

In [0]:
import pandas as pd
data = pd.read_csv('pristavki.csv', header=None, names=['text'])


In [0]:
marked = []

for text in data.text.values[:1000]:
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

Этот метод позволяет находить даже те игры, которых не было ни в тренировочной выборке, ни в тестовых:

In [38]:
ner_model(['Killzone 3', 'Need for Speed: Rivals'])

[[['Killzone', '3'], ['Need', 'for', 'Speed', ':', 'Rivals']],
 [['B-GM', 'I-GM'], ['B-GM', 'I-GM', 'I-GM', 'O', 'B-GM']]]

Однако есть проблемы с тем, чтоб находить игры по переведенному названию, а также тем, чтобы находить некоторые из игр вне контекста:

In [39]:
ner_model(['Farcry primal 1500 и За гранью 1500.', 'Farcry primal'])

[[['Farcry', 'primal', '1500', 'и', 'За', 'гранью', '1500', '.'],
  ['Farcry', 'primal']],
 [['B-GM', 'B-GM', 'I-GM', 'O', 'O', 'O', 'O', 'O'], ['B-GM', 'O']]]

Некоторые из сабтайтлов у игр также не распознаются как части названия игр:

In [41]:
ner_model(['Assassin\'s Creed Syndicate', 'продам Assassin\'s Creed Syndicate', 'Watch Dogs 2 Deluxe Edition', 'продам Watch Dogs 2 Deluxe Edition'])

[[["Assassin's", 'Creed', 'Syndicate'],
  ['продам', "Assassin's", 'Creed', 'Syndicate'],
  ['Watch', 'Dogs', '2', 'Deluxe', 'Edition'],
  ['продам', 'Watch', 'Dogs', '2', 'Deluxe', 'Edition']],
 [['B-GM', 'I-GM', 'B-GM'],
  ['O', 'B-GM', 'I-GM', 'B-GM'],
  ['B-GM', 'I-GM', 'I-GM', 'O', 'O'],
  ['O', 'B-GM', 'I-GM', 'O', 'O', 'O']]]